## import packages

In [30]:
!pip install opencv-python matplotlib imutils sklearn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [31]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, save_model, Sequential
from tensorflow.keras.applications import ResNet50V2, VGG16
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam

## data preprocessing

In [33]:
# from minio import Minio
# from minio.error import ResponseError

# minioClient = Minio(
#                 'minio-service.kubeflow:9000',
#                 access_key='minio', 
#                 secret_key='minio123', 
#                 secure=False
#             )

# import os
# import io
# from PIL import Image

# data = []
# labels = []

# for i in minioClient.list_objects('dataset', prefix='covid-19', recursive=True):
#     label = i.object_name.split(os.path.sep)[-2]

#     o = minioClient.get_object('dataset', i.object_name)
#     image = np.fromstring(o.read(), dtype = np.uint8)

#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (224, 224))

#     data.append(image)
#     labels.append(label)

In [34]:
imagePath = "./all/train"
imagePaths = list(paths.list_images(imagePath))
data = []
labels = []

for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[-2]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    labels.append(label)

In [35]:
data = np.array(data) / 255.0
labels = np.array(labels)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(integer_encoded)

(x_train, x_val, y_train, y_val) = train_test_split(data, labels, test_size=0.20, stratify=labels)

## AI model training

In [36]:
model = Sequential()
adam_s = Adam(learning_rate = 0.00001)

#model.add(VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='average'))
model.add(ResNet50V2(input_shape=(224, 224, 3),include_top=False, weights='imagenet',pooling='average'))

for layer in model.layers:
    layer.trainable = False

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam_s, metrics=['accuracy'])


H = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=8)
#H = model.fit(data, labels, epochs=50, validation_split=0.2, batch_size=8)

94674944/94668760 [==============================] - 8s 0us/step
Train on 160 samples, validate on 40 samples
Epoch 1/10
160/160 [==============================] - 16s 99ms/sample - loss: 0.9661 - accuracy: 0.5500 - val_loss: 1.4654 - val_accuracy: 0.5750
Epoch 2/10
160/160 [==============================] - 12s 76ms/sample - loss: 0.3640 - accuracy: 0.8813 - val_loss: 1.2304 - val_accuracy: 0.5750
Epoch 3/10
160/160 [==============================] - 12s 76ms/sample - loss: 0.1386 - accuracy: 0.9875 - val_loss: 1.0387 - val_accuracy: 0.6500
Epoch 4/10
160/160 [==============================] - 12s 75ms/sample - loss: 0.0779 - accuracy: 0.9937 - val_loss: 0.9781 - val_accuracy: 0.6500
Epoch 5/10
160/160 [==============================] - 12s 75ms/sample - loss: 0.0600 - accuracy: 0.9937 - val_loss: 0.9315 - val_accuracy: 0.6500
Epoch 6/10
160/160 [==============================] - 12s 75ms/sample - loss: 0.0333 - accuracy: 1.0000 - val_loss: 0.8736 - val_accuracy: 0.6500
Epoch 7/10
160

## save AI model

In [41]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',	# Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0' 	# If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

save_model(model, "s3://model/covid/1")

INFO:tensorflow:Assets written to: s3://model/covid/1/assets
